# VAE encoder network

In [1]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2                # dimensionality of latent space: 2D plane

input_img = keras.Input(shape = img_shape)

Using TensorFlow backend.


### Add the layers

In [3]:
x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2,2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

# Shape
shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)

(None, 14, 14, 64)


### Encode into latent dimension

In [4]:
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

# The input image ends up being encoded into 2 parameters
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Latent space sampling function

In [5]:
def sampling(args):
    z_mean, z_log_var = args
    
    # draw epsilon from normal distribution
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# VAE decoder network, mapping latent space points to images

In [6]:
decoder_input = layers.Input(K.int_shape(z)[1:]) # 1st dim must be batch size

# upsample the input
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)

x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)

### Build model

In [ ]:
decoder = Model(decoder_input, x)
z